In [ ]:
"""
carbon_concrete_explorer.ipynb - Advanced Carbon Negative Concrete Analysis
Copyright 2024 Cette AI

Enhanced analysis framework for carbon-negative concrete incorporating:
- Multi-objective optimization
- Uncertainty quantification
- Thermodynamic modeling
- Life cycle analysis
- Economic modeling with carbon credits
- Advanced mixture design

Author: Michael R. Lafave
Last Modified: 2024-11-30
"""

import tensorflow as tf
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from typing import Dict, List, Tuple, Optional
from dataclasses import dataclass
import tensorflow_probability as tfp

@dataclass
class ConcreteComposition:
    """Enhanced concrete composition with uncertainty quantification."""
    
    component: str
    fraction: float
    uncertainty: float
    min_bound: float
    max_bound: float
    cost_per_kg: float
    co2_per_kg: float
    sequestration_rate: float
    
    def validate(self) -> bool:
        """Validate composition constraints."""
        return (
            self.min_bound <= self.fraction <= self.max_bound and
            self.uncertainty >= 0 and
            self.fraction + self.uncertainty <= self.max_bound
        )

class MaterialDatabase:
    """Enhanced materials database with property prediction."""
    
    def __init__(self):
        # Initialize optimal composition with uncertainties
        self.compositions = {
            # Aluminosilicate Materials
            'fly_ash_f': ConcreteComposition(
                'fly_ash_f', 0.30, 0.02, 0.20, 0.40, 0.02, 0.01, 0.0
            ),
            'ggbfs': ConcreteComposition(
                'ggbfs', 0.20, 0.02, 0.15, 0.25, 0.05, 0.05, 0.0
            ),
            'metakaolin': ConcreteComposition(
                'metakaolin', 0.05, 0.01, 0.03, 0.07, 0.20, 0.15, 0.0
            ),
            'silica_fume': ConcreteComposition(
                'silica_fume', 0.03, 0.005, 0.02, 0.05, 0.30, 0.02, 0.0
            ),
            
            # CO2 Absorbing Materials
            'magnesium_silicate': ConcreteComposition(
                'magnesium_silicate', 0.15, 0.02, 0.10, 0.20, 0.10, 0.05, 0.30
            ),
            'olivine': ConcreteComposition(
                'olivine', 0.15, 0.02, 0.10, 0.20, 0.05, 0.05, 0.30
            ),
            'biochar': ConcreteComposition(
                'biochar', 0.10, 0.01, 0.05, 0.15, 0.50, 0.00, 2.00
            ),
            'basalt_dust': ConcreteComposition(
                'basalt_dust', 0.02, 0.005, 0.01, 0.03, 0.02, 0.01, 0.00
            ),
            
            # Carbonates
            'calcium_carbonate': ConcreteComposition(
                'calcium_carbonate', 0.03, 0.005, 0.02, 0.05, 0.05, 0.05, 0.10
            ),
            'magnesium_carbonate': ConcreteComposition(
                'magnesium_carbonate', 0.02, 0.005, 0.01, 0.03, 0.05, 0.05, 0.10
            ),
            
            # Advanced Components
            'algae_silica': ConcreteComposition(
                'algae_silica', 0.01, 0.002, 0.005, 0.02, 1.00, 0.00, 1.00
            ),
            'algae_biomass': ConcreteComposition(
                'algae_biomass', 0.03, 0.005, 0.02, 0.05, 1.00, 0.00, 1.00
            ),
            'alginate_beads': ConcreteComposition(
                'alginate_beads', 0.05, 0.01, 0.03, 0.07, 1.00, 0.00, 1.00
            ),
            'csh_seeds': ConcreteComposition(
                'csh_seeds', 0.02, 0.005, 0.01, 0.03, 0.50, 0.50, 0.50
            )
        }

class CarbonNegativeConcrete:
    """Enhanced analyzer for carbon negative concrete formulation."""
    
    def __init__(
        self,
        batch_size_tonnes: float = 1000.0,
        temperature: float = 298.15,
        relative_humidity: float = 0.65
    ):
        self.batch_size = batch_size_tonnes
        self.temperature = temperature
        self.relative_humidity = relative_humidity
        
        self.material_db = MaterialDatabase()
        self.property_model = self._build_property_model()
        self.optimization_model = self._build_optimization_model()
        
    def _build_property_model(self) -> tf.keras.Model:
        """Build deep learning model for concrete property prediction."""
        
        # Input layers for composition and conditions
        composition_input = tf.keras.Input(shape=(len(self.material_db.compositions),))
        condition_input = tf.keras.Input(shape=(2,))  # Temperature and RH
        
        # Combine inputs
        x = tf.keras.layers.Concatenate()([composition_input, condition_input])
        
        # Dense layers with physics-informed constraints
        x = tf.keras.layers.Dense(256, activation='swish')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        x = tf.keras.layers.Dense(128, activation='swish')(x)
        x = tf.keras.layers.Dropout(0.2)(x)
        
        # Multiple output heads
        strength = tf.keras.layers.Dense(1, name='strength')(x)
        workability = tf.keras.layers.Dense(1, name='workability')(x)
        durability = tf.keras.layers.Dense(1, name='durability')(x)
        co2_uptake = tf.keras.layers.Dense(1, name='co2_uptake')(x)
        
        model = tf.keras.Model(
            inputs=[composition_input, condition_input],
            outputs=[strength, workability, durability, co2_uptake]
        )
        
        # Custom loss incorporating physical constraints
        def physics_constrained_loss(y_true: tf.Tensor, y_pred: tf.Tensor) -> tf.Tensor:
            mse = tf.keras.losses.MSE(y_true, y_pred)
            # Add physics-based penalties
            physical_constraint = tf.reduce_mean(tf.maximum(0.0, -y_pred))
            return mse + 0.1 * physical_constraint
        
        model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
            loss={
                'strength': physics_constrained_loss,
                'workability': physics_constrained_loss,
                'durability': physics_constrained_loss,
                'co2_uptake': physics_constrained_loss
            },
            metrics=['mae']
        )
        
        return model
    
    def _build_optimization_model(self) -> tfp.optimizer.LBFGSBOptimizer:
        """Build Bayesian optimization model for composition optimization."""
        return tfp.optimizer.LBFGSBOptimizer(
            num_correction_pairs=10,
            tolerance=1e-8,
            max_iterations=1000
        )
    
    def calculate_costs(self, composition: Dict[str, float]) -> pd.DataFrame:
        """Calculate detailed cost breakdown with uncertainty."""
        
        costs = []
        for material, fraction in composition.items():
            mat_data = self.material_db.compositions[material]
            quantity = fraction * self.batch_size * 1000  # kg
            
            # Calculate costs with uncertainty
            base_cost = quantity * mat_data.cost_per_kg
            cost_uncertainty = base_cost * mat_data.uncertainty
            
            costs.append({
                'Material': material,
                'Quantity_kg': quantity,
                'Unit_Cost': mat_data.cost_per_kg,
                'Base_Cost': base_cost,
                'Cost_Uncertainty': cost_uncertainty,
                'Total_Cost_Min': base_cost - cost_uncertainty,
                'Total_Cost_Max': base_cost + cost_uncertainty
            })
        
        return pd.DataFrame(costs)
    
    def calculate_emissions(self, composition: Dict[str, float]) -> pd.DataFrame:
        """Calculate detailed emissions analysis with uncertainty."""
        
        emissions = []
        for material, fraction in composition.items():
            mat_data = self.material_db.compositions[material]
            quantity = fraction * self.batch_size * 1000
            
            # Calculate emissions with uncertainty
            base_emissions = quantity * mat_data.co2_per_kg
            emission_uncertainty = base_emissions * mat_data.uncertainty
            
            emissions.append({
                'Material': material,
                'Quantity_kg': quantity,
                'Emission_Factor': mat_data.co2_per_kg,
                'Base_Emissions': base_emissions,
                'Emission_Uncertainty': emission_uncertainty,
                'Total_Emissions_Min': base_emissions - emission_uncertainty,
                'Total_Emissions_Max': base_emissions + emission_uncertainty
            })
        
        return pd.DataFrame(emissions)
    
    def calculate_sequestration(self, composition: Dict[str, float]) -> pd.DataFrame:
        """Calculate detailed CO2 sequestration analysis with uncertainty."""
        
        sequestration = []
        for material, fraction in composition.items():
            mat_data = self.material_db.compositions[material]
            quantity = fraction * self.batch_size * 1000
            
            if mat_data.sequestration_rate > 0:
                # Calculate sequestration with uncertainty
                base_sequestration = quantity * mat_data.sequestration_rate
                seq_uncertainty = base_sequestration * mat_data.uncertainty
                
                # Add temperature and humidity effects
                temp_factor = self._temperature_effect(self.temperature)
                rh_factor = self._humidity_effect(self.relative_humidity)
                
                adjusted_sequestration = base_sequestration * temp_factor * rh_factor
                
                sequestration.append({
                    'Material': material,
                    'Quantity_kg': quantity,
                    'Sequestration_Rate': mat_data.sequestration_rate,
                    'Temperature_Factor': temp_factor,
                    'Humidity_Factor': rh_factor,
                    'Base_Sequestration': base_sequestration,
                    'Adjusted_Sequestration': adjusted_sequestration,
                    'Sequestration_Uncertainty': seq_uncertainty,
                    'Total_Sequestration_Min': adjusted_sequestration - seq_uncertainty,
                    'Total_Sequestration_Max': adjusted_sequestration + seq_uncertainty
                })
        
        return pd.DataFrame(sequestration)

    def _temperature_effect(self, temperature: float) -> float:
        """Calculate temperature effect on CO2 sequestration."""
        # Arrhenius-type relationship
        R = 8.314  # Gas constant
        Ea = 50000  # Activation energy J/mol
        T_ref = 298.15  # Reference temperature
        
        return np.exp(-Ea/R * (1/temperature - 1/T_ref))
    
    def _humidity_effect(self, relative_humidity: float) -> float:
        """Calculate humidity effect on CO2 sequestration."""
        # Simplified model based on experimental data
        return 0.2 + 0.8 * relative_humidity

    def calculate_net_impact(
        self,
        carbon_price: float = 60.0  # $/tonne CO2
    ) -> Dict[str, float]:
        """Calculate comprehensive net impact analysis."""
        
        # Get base calculations
        costs_df = self.calculate_costs(self.material_db.compositions)
        emissions_df = self.calculate_emissions(self.material_db.compositions)
        sequestration_df = self.calculate_sequestration(self.material_db.compositions)
        
        # Calculate totals with uncertainty
        total_cost = costs_df['Base_Cost'].sum()
        cost_uncertainty = np.sqrt(np.sum(costs_df['Cost_Uncertainty']**2))
        
        total_emissions = emissions_df['Base_Emissions'].sum()
        emissions_uncertainty = np.sqrt(np.sum(emissions_df['Emission_Uncertainty']**2))
        
        total_sequestration = sequestration_df['Adjusted_Sequestration'].sum()
        sequestration_uncertainty = np.sqrt(
            np.sum(sequestration_df['Sequestration_Uncertainty']**2)
        )
        
        # Calculate net CO2 impact
        net_co2 = total_emissions - total_sequestration
        net_co2_uncertainty = np.sqrt(emissions_uncertainty**2 + sequestration_uncertainty**2)
        
        # Calculate carbon credits
        carbon_credit = abs(min(0, net_co2)) * carbon_price / 1000  # Convert to tonnes
        credit_uncertainty = net_co2_uncertainty * carbon_price / 1000
        
        return {
            'Total_Cost': {
                'value': total_cost,
                'uncertainty': cost_uncertainty
            },
            'Total_Emissions': {
                'value': total_emissions,
                'uncertainty': emissions_uncertainty
            },
            'Total_Sequestration': {
                'value': total_sequestration,
                'uncertainty': sequestration_uncertainty
            },
            'Net_CO2': {
                'value': net_co2,
                'uncertainty': net_co2_uncertainty
            },
            'Carbon_Credit': {
                'value': carbon_credit,
                'uncertainty': credit_uncertainty
            },
            'Final_Cost': {
                'value': total_cost - carbon_credit,
                'uncertainty': np.sqrt(cost_uncertainty**2 + credit_uncertainty**2)
            }
        }

    def optimize_composition(
        self,
        target_strength: float = 40.0,  # MPa
        max_cost: float = 100.0,  # $/tonne
        min_co2_reduction: float = 300.0  # kg/tonne
    ) -> Dict[str, float]:
        """Optimize concrete composition using Bayesian optimization."""
        
        def objective(x: tf.Tensor) -> tf.Tensor:
            """Objective function for optimization."""
            # Convert parameters to composition
            composition = self._params_to_composition(x)
            
            # Predict properties
            properties = self.property_model.predict([
                tf.expand_dims(x, 0),
                tf.constant([[self.temperature, self.relative_humidity]])
            ])
            
            strength = properties[0][0]
            workability = properties[1][0]
            co2_uptake = properties[3][0]
            
            # Calculate costs
            costs = self.calculate_costs(composition)
            total_cost = costs['Base_Cost'].sum()
            # Calculate penalties
            strength_penalty = tf.abs(strength - target_strength)
            cost_penalty = tf.maximum(0.0, total_cost - max_cost)
            co2_penalty = tf.maximum(0.0, min_co2_reduction - co2_uptake)
            
            # Add composition constraint penalties
            composition_sum = tf.reduce_sum(x)
            sum_penalty = tf.abs(composition_sum - 1.0) * 1000
            
            # Combine objectives with weights
            return (
                strength_penalty * 10.0 +
                cost_penalty * 0.1 +
                co2_penalty * 1.0 +
                sum_penalty
            )
        
        # Initial composition
        initial_x = tf.constant([
            comp.fraction for comp in self.material_db.compositions.values()
        ])
        
        # Optimization bounds
        bounds = [(comp.min_bound, comp.max_bound) 
                 for comp in self.material_db.compositions.values()]
        
        # Run optimization
        result = self.optimization_model.minimize(
            objective,
            initial_position=initial_x,
            lower_bound=[b[0] for b in bounds],
            upper_bound=[b[1] for b in bounds]
        )
        
        return self._params_to_composition(result.position)
    
    def _params_to_composition(self, params: tf.Tensor) -> Dict[str, float]:
        """Convert optimization parameters to composition dictionary."""
        return {
            name: float(params[i])
            for i, name in enumerate(self.material_db.compositions.keys())
        }

    def analyze_lifecycle_impact(
        self,
        service_life: int = 50,  # years
        transport_distance: float = 100.0,  # km
        maintenance_interval: int = 10  # years
    ) -> Dict[str, Dict[str, float]]:
        """Perform detailed lifecycle impact analysis."""
        
        impacts = {}
        
        # Production phase impacts
        production = self.calculate_net_impact()
        impacts['Production'] = production
        
        # Transportation impacts
        transport_emissions = (
            self.batch_size *  # tonnes
            transport_distance *  # km
            0.062  # kg CO2/tonne-km (typical truck transport)
        )
        
        impacts['Transportation'] = {
            'value': transport_emissions,
            'uncertainty': transport_emissions * 0.2  # 20% uncertainty
        }
        
        # Use phase impacts
        n_maintenance = service_life // maintenance_interval
        maintenance_impact = production['Net_CO2']['value'] * 0.1  # 10% of initial
        
        impacts['Maintenance'] = {
            'value': maintenance_impact * n_maintenance,
            'uncertainty': maintenance_impact * n_maintenance * 0.3  # 30% uncertainty
        }
        
        # End of life impacts/benefits
        eol_sequestration = production['Total_Sequestration']['value'] * 0.2  # Additional 20%
        
        impacts['End_of_Life'] = {
            'value': -eol_sequestration,  # Negative because it's beneficial
            'uncertainty': eol_sequestration * 0.4  # 40% uncertainty
        }
        
        # Calculate total lifecycle impact
        total_value = sum(phase['value'] for phase in impacts.values())
        total_uncertainty = np.sqrt(sum(phase['uncertainty']**2 for phase in impacts.values()))
        
        impacts['Total_Lifecycle'] = {
            'value': total_value,
            'uncertainty': total_uncertainty
        }
        
        return impacts

    def generate_report(self) -> str:
        """Generate comprehensive analysis report."""
        
        # Get all analyses
        composition = self.optimize_composition()
        net_impact = self.calculate_net_impact()
        lifecycle = self.analyze_lifecycle_impact()
        
        # Create report
        report = [
            "Carbon Negative Concrete Analysis Report",
            "=====================================",
            "",
            "1. Optimized Composition",
            "---------------------"
        ]
        
        for material, fraction in composition.items():
            report.append(f"{material}: {fraction:.3f}")
        
        report.extend([
            "",
            "2. Environmental Impact",
            "--------------------",
            f"Total CO2 Emissions: {net_impact['Total_Emissions']['value']:.1f} ± "
            f"{net_impact['Total_Emissions']['uncertainty']:.1f} kg",
            f"Total CO2 Sequestration: {net_impact['Total_Sequestration']['value']:.1f} ± "
            f"{net_impact['Total_Sequestration']['uncertainty']:.1f} kg",
            f"Net CO2 Impact: {net_impact['Net_CO2']['value']:.1f} ± "
            f"{net_impact['Net_CO2']['uncertainty']:.1f} kg",
            "",
            "3. Economic Analysis",
            "------------------",
            f"Base Cost: ${net_impact['Total_Cost']['value']:.2f} ± "
            f"${net_impact['Total_Cost']['uncertainty']:.2f}",
            f"Carbon Credits: ${net_impact['Carbon_Credit']['value']:.2f} ± "
            f"${net_impact['Carbon_Credit']['uncertainty']:.2f}",
            f"Final Cost: ${net_impact['Final_Cost']['value']:.2f} ± "
            f"${net_impact['Final_Cost']['uncertainty']:.2f}",
            "",
            "4. Lifecycle Analysis",
            "------------------"
        ])
        
        for phase, impact in lifecycle.items():
            report.append(
                f"{phase}: {impact['value']:.1f} ± {impact['uncertainty']:.1f} kg CO2"
            )
        
        return "\n".join(report)

# Visualization functions
def plot_composition_breakdown(analyzer: CarbonNegativeConcrete) -> go.Figure:
    """Create interactive composition breakdown visualization."""
    
    composition = analyzer.optimize_composition()
    
    # Group materials by category
    categories = {
        'Aluminosilicates': ['fly_ash_f', 'ggbfs', 'metakaolin', 'silica_fume'],
        'CO2 Absorbing': ['magnesium_silicate', 'olivine', 'biochar', 'basalt_dust'],
        'Carbonates': ['calcium_carbonate', 'magnesium_carbonate'],
        'Advanced Components': ['algae_silica', 'algae_biomass', 'alginate_beads', 'csh_seeds']
    }
    
    # Prepare data
    data = []
    for category, materials in categories.items():
        for material in materials:
            data.append({
                'Category': category,
                'Material': material,
                'Fraction': composition[material]
            })
    
    df = pd.DataFrame(data)
    
    # Create sunburst chart
    fig = px.sunburst(
        df,
        path=['Category', 'Material'],
        values='Fraction',
        title='Concrete Composition Breakdown'
    )
    
    return fig

def plot_emissions_analysis(analyzer: CarbonNegativeConcrete) -> go.Figure:
    """Create interactive emissions vs sequestration visualization."""
    
    net_impact = analyzer.calculate_net_impact()
    
    # Create Sankey diagram
    fig = go.Figure(data=[go.Sankey(
        node = dict(
            pad = 15,
            thickness = 20,
            line = dict(color = "black", width = 0.5),
            label = ["Emissions", "Sequestration", "Net Impact"],
            color = ["red", "green", "blue"]
        ),
        link = dict(
            source = [0, 1],
            target = [2, 2],
            value = [
                net_impact['Total_Emissions']['value'],
                -net_impact['Total_Sequestration']['value']
            ]
        )
    )])
    
    fig.update_layout(title_text="CO2 Emissions and Sequestration Flow")
    return fig

def plot_lifecycle_analysis(analyzer: CarbonNegativeConcrete) -> go.Figure:
    """Create interactive lifecycle analysis visualization."""
    
    lifecycle = analyzer.analyze_lifecycle_impact()
    
    # Prepare data
    phases = list(lifecycle.keys())
    values = [impact['value'] for impact in lifecycle.values()]
    uncertainties = [impact['uncertainty'] for impact in lifecycle.values()]
    
    # Create waterfall chart
    fig = go.Figure(go.Waterfall(
        name = "Lifecycle CO2",
        orientation = "v",
        measure = ["relative"] * (len(phases) - 1) + ["total"],
        x = phases,
        y = values,
        error_y = dict(
            type='data',
            array=uncertainties,
            visible=True
        ),
        connector = {"line":{"color":"rgb(63, 63, 63)"}}
    ))
    
    fig.update_layout(
        title = "Lifecycle CO2 Impact Analysis",
        showlegend = True
    )
    
    return fig

# Example usage
if __name__ == "__main__":
    # Initialize analyzer
    analyzer = CarbonNegativeConcrete(
        batch_size_tonnes=1000.0,
        temperature=298.15,
        relative_humidity=0.65
    )
    
    # Generate and print report
    report = analyzer.generate_report()
    print(report)
    
    # Create visualizations
    composition_fig = plot_composition_breakdown(analyzer)
    emissions_fig = plot_emissions_analysis(analyzer)
    lifecycle_fig = plot_lifecycle_analysis(analyzer)
    
    # Show figures
    composition_fig.show()
    emissions_fig.show()
    lifecycle_fig.show()